# Worksheet 08

Name:  Quan Ho
UID: U17207751

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [19]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.064623523154372, 5.362550058497434, 5.914131221574851, 7.252923895609808, 4.044314458543299, 4.908588492016687, 5.269443097207914, 5.798813470826919, 5.144337798116265, 4.117377037269324]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [20]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[8.468765820793188, 8.369569365324576, 8.478700012415024, 8.71784279728441, 6.626996947174047, 6.690983356115752, 6.440666269811988, 9.096122536027625, 6.986175629749794, 6.7852092831667825]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [21]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1[-1]
        s1.remove(p1)
        data.append(p1)
    else:
        p2 = s2[-1]
        s2.remove(p2)
        data.append(p2)
print(data)

[4.117377037269324, 6.7852092831667825, 6.986175629749794, 9.096122536027625, 6.440666269811988, 5.144337798116265, 5.798813470826919, 6.690983356115752, 5.269443097207914, 4.908588492016687]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Prior: 0.5, during coin flip. Next is mean/variances

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [22]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) ,sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.7852092831667825, 6.986175629749794, 9.096122536027625, 6.440666269811988, 6.690983356115752]
[4.117377037269324, 5.144337798116265, 5.798813470826919, 5.269443097207914, 4.908588492016687]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.199831414974389,  mean_2 = 5.047711979087422
var_1 = 0.9297476774327229,  var_2 = 4.9331246937547695


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


The mean is close but the variance is very far from C2.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [23]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]*prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1]*prob_s[1] / prob_x  )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.117377037269324
probability of observing that point if it came from cluster 0 =  0.0017609105086226569
probability of observing that point if it came from cluster 1 =  0.07944469893282664
point =  6.7852092831667825
probability of observing that point if it came from cluster 0 =  0.3884725592468767
probability of observing that point if it came from cluster 1 =  0.07600644204121575
point =  6.986175629749794
probability of observing that point if it came from cluster 0 =  0.41790528967226487
probability of observing that point if it came from cluster 1 =  0.07486151262266777
point =  9.096122536027625
probability of observing that point if it came from cluster 0 =  0.053609350470984495
probability of observing that point if it came from cluster 1 =  0.057748881732528894
point =  6.440666269811988
probability of observing that point if it came from cluster 0 =  0.30744606862744617
probability of observing that point if it came from cluster 1 =  0.07770957127231037
point =  5.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [27]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([(x[0]-mean[0])**2 * x[1] for x in zip(data,prob_s0_x)]) , sum([(x[0]-mean[1])**2 * x[1] for x in zip(data,prob_s1_x)]) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5346614016256412,  P(S_2) = 0.4653385983743587
mean_1 = 6.555453069052082,  mean_2 = 5.62778153368141
var_1 = 5.634840971710582,  var_2 = 9.959378576183592


The probability of seeing s1 and s2 are similimiar. They are different than the K-means because GMM is a proabilisitc model as opposed to K-means non-probalisitc distance=based clustering alogrithm. Meaning that GMM assigns probabilites to each cluster while K-means assigns each point to a single cluster.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [28]:
print('For S1:')
print(prob_s0_x)
print('For S2:')
print(prob_s1_x)

For S1:
[0.021684591997210546, 0.8363619413785449, 0.8480792288075824, 0.4814134474855026, 0.7982385217245277, 0.3154353011026991, 0.6329892770527831, 0.8284282505958779, 0.3809349833480466, 0.20304847276363766]
For S2:
[0.9783154080027894, 0.163638058621455, 0.15192077119241765, 0.5185865525144974, 0.20176147827547233, 0.6845646988973009, 0.36701072294721687, 0.1715717494041221, 0.6190650166519533, 0.7969515272363623]


The main difference  in the loop between data points is how they are assigned to clusters based on these updated probabilities. Some data points (s_0 or s_1) will be more confidently assigned to one cluster , while some other are assigned with less confidence.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [31]:
hard_assignments = []  # List to store the hard cluster assignments (0 or 1)

for p_c0, p_c1 in zip(prob_s0_x, prob_s1_x):
    if p_c0 > p_c1:
        hard_assignments.append(0)
    else:
        hard_assignments.append(1)
hard_assignments


[1, 0, 0, 1, 0, 1, 0, 0, 1, 1]

In [32]:
print(data)

[4.117377037269324, 6.7852092831667825, 6.986175629749794, 9.096122536027625, 6.440666269811988, 5.144337798116265, 5.798813470826919, 6.690983356115752, 5.269443097207914, 4.908588492016687]
